In [18]:
import findspark
findspark.init()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("corrupt data handling").getOrCreate()

filepath="/Users/bindu/Documents/sales.txt"
df=spark.read.format("csv")\
.option("header","true")\
.option("delimiter","|")\
.load(filepath)

In [19]:
df.show()

+----+----+----------+------+----+
|  id|name|  sales_dt|amount|flag|
+----+----+----------+------+----+
|   1| bin|01-01-2024|  1000|   y|
| 2tt|   a|03-01-2024|  1320|   y|
|   3|   b|04-01-2024|  2000|   y|
|   4|   d|05-01-2024|  3000|   n|
|5qwe|   f|06-01-2024|  4000|   n|
|   6|   r|07-01-2024|  5000|   n|
+----+----+----------+------+----+



In [24]:
#Data in id 2 and 5 are corrupted
#handle by passing those data as null

from pyspark.sql.types import *
schema=StructType([
StructField('id',IntegerType()),
StructField('name',StringType()),
StructField('sales_dt',StringType()),
StructField('amount',IntegerType()),
StructField('flag',StringType())])

dfschema=spark.read.format("csv")\
.option("header","true")\
.option("delimiter","|")\
.schema(schema)\
.load(filepath)



In [25]:
dfschema.show()

+----+----+----------+------+----+
|  id|name|  sales_dt|amount|flag|
+----+----+----------+------+----+
|   1| bin|01-01-2024|  1000|   y|
|null|   a|03-01-2024|  1320|   y|
|   3|   b|04-01-2024|  2000|   y|
|   4|   d|05-01-2024|  3000|   n|
|null|   f|06-01-2024|  4000|   n|
|   6|   r|07-01-2024|  5000|   n|
+----+----+----------+------+----+



In [28]:
#remove the corrupted data column value alone using permissive option
dfpermissive=spark.read.format("csv")\
.option("header","true")\
.option("delimiter","|")\
.option('mode','PERMISSIVE').schema(schema)\
.load(filepath)

dfpermissive.show()


+----+----+----------+------+----+
|  id|name|  sales_dt|amount|flag|
+----+----+----------+------+----+
|   1| bin|01-01-2024|  1000|   y|
|null|   a|03-01-2024|  1320|   y|
|   3|   b|04-01-2024|  2000|   y|
|   4|   d|05-01-2024|  3000|   n|
|null|   f|06-01-2024|  4000|   n|
|   6|   r|07-01-2024|  5000|   n|
+----+----+----------+------+----+



In [29]:
#remove the corrupted data row itself using dropmalformed option
dfdrop=spark.read.format("csv")\
.option("header","true")\
.option("delimiter","|")\
.option('mode','DROPMALFORMED').schema(schema)\
.load(filepath)

dfdrop.show()

+---+----+----------+------+----+
| id|name|  sales_dt|amount|flag|
+---+----+----------+------+----+
|  1| bin|01-01-2024|  1000|   y|
|  3|   b|04-01-2024|  2000|   y|
|  4|   d|05-01-2024|  3000|   n|
|  6|   r|07-01-2024|  5000|   n|
+---+----+----------+------+----+



In [30]:
#fail the session itself
dfp=spark.read.format("csv")\
.option("header","true")\
.option("delimiter","|")\
.option('mode','FAILFAST').schema(schema)\
.load(filepath)

dfp.show()

Py4JJavaError: An error occurred while calling o165.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (host.docker.internal executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$10.nextCur(Iterator.scala:587)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:601)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:576)

In [41]:
#copy the corrupted data to other file

schemaE=StructType([
StructField('id',IntegerType()),
StructField('name',StringType()),
StructField('sales_dt',StringType()),
StructField('amount',IntegerType()),
StructField('flag',StringType()),
StructField('errorRecords',StringType())])


dfc=spark.read.format('csv').option("header","true").option("delimiter","|").option('mode','PERMISSIVE').schema(schemaE)\
.option('columnNameOfCorruptRecord','errorRecords')\
.load(filepath)

dfc.show()


+----+----+----------+------+----+--------------------+
|  id|name|  sales_dt|amount|flag|        errorRecords|
+----+----+----------+------+----+--------------------+
|   1| bin|01-01-2024|  1000|   y|                null|
|null|   a|03-01-2024|  1320|   y|2tt|a|03-01-2024|...|
|   3|   b|04-01-2024|  2000|   y|                null|
|   4|   d|05-01-2024|  3000|   n|                null|
|null|   f|06-01-2024|  4000|   n|5qwe|f|06-01-2024...|
|   6|   r|07-01-2024|  5000|   n|                null|
+----+----+----------+------+----+--------------------+



In [45]:
#filter the non corrupt data
df_non_error=dfc.filter(col('errorRecords').isNull())
df_non_error.show()

+---+----+----------+------+----+------------+
| id|name|  sales_dt|amount|flag|errorRecords|
+---+----+----------+------+----+------------+
|  1| bin|01-01-2024|  1000|   y|        null|
|  3|   b|04-01-2024|  2000|   y|        null|
|  4|   d|05-01-2024|  3000|   n|        null|
|  6|   r|07-01-2024|  5000|   n|        null|
+---+----+----------+------+----+------------+



In [46]:
#filter the corrupt data
df_non_error=dfc.filter(col('errorRecords').isNotNull())
df_non_error.show()

+----+----+----------+------+----+--------------------+
|  id|name|  sales_dt|amount|flag|        errorRecords|
+----+----+----------+------+----+--------------------+
|null|   a|03-01-2024|  1320|   y|2tt|a|03-01-2024|...|
|null|   f|06-01-2024|  4000|   n|5qwe|f|06-01-2024...|
+----+----+----------+------+----+--------------------+

